In [13]:
#dependencies
import h5py
import numpy as np

In [14]:
import os 

In [15]:
pwd

'/Users/layalmahfoud/Desktop/PostBoettgerLayalJordan'

In [16]:
#importing bin file from a Kr 1e-X2C with no 2e-SOC (or approx.) with dyall-ae4z basis set.
file_NeX2cNoscal = h5py.File('./data/x2c1enoscal/Ne_dyall_ae4z/neon_x2c_noscal.bin','r')

In [17]:
#importing fock matrices in pauli spinor basis in AO basis 
#only need spin-dependent part -- no scalar for now. 
fockx_NeX2cNoscal = np.array(file_NeX2cNoscal['SCF/FOCK_MX'][:])
focky_NeX2cNoscal = np.array(file_NeX2cNoscal['SCF/FOCK_MY'][:])
fockz_NeX2cNoscal = np.array(file_NeX2cNoscal['SCF/FOCK_MZ'][:])

In [18]:
#sanity check 

#2c fock matrix traditionally has dimensionality (2*NBasis, 2*NBasis)
#but when we spin scatter it (= put in pauli spinor basis = split it up into X,Y,Z,S)...
#each matrix now has dimension (NBasis, NBasis)

#From output file:


#rows
print("number of fock_mx rows: " + str(len(fockx_NeX2cNoscal)))

#columns
print("number of fock_mx columns: " + str(len(fockx_NeX2cNoscal[0])))

#rows = columns = NBasis. That's good!

number of fock_mx rows: 103
number of fock_mx columns: 103


In [19]:
#save dimension of 2c fock matrix in pauli spinor basis = NBasis
NBasis =len(fockx_NeX2cNoscal)

In [20]:
print(NBasis )

103


### We want to convert our spin-scattered (S,X,Y,Z) Fock matrices into the one total Fock matrix.
### cite: the breit paper, the gaunt paper -- but I can set this up for you. 
### This will make life easier for us. 
# For two-component calculations... 
# $$
F_{total} = \frac{1}{2} 
\begin{pmatrix}
F_s + F_z & F_x-iF_y \\
F_x + iF_y & F_s-Fz 
\end{pmatrix}
$$ 
### That is how to make the total Fock matrix out of (S,X,Y,Z). 
# However,
### we don't need the total Fock matrix, we need the spin-dependent Fock matrix
# $$
F_{total} = F_{spin-free} + F_{spin-dependent}
$$
# $$
F_{spin-free} = 
\frac{1}{2}\begin{pmatrix}
F_s & 0 \\ 
0   & F_s
\end{pmatrix}
$$
## So, with a little algebra...
# $$
\frac{1}{2} 
\begin{pmatrix}
F_s + F_z & F_x-iF_y \\
F_x + iF_y & F_s-Fz 
\end{pmatrix}
=
\frac{1}{2}
\begin{pmatrix}
F_s & 0 \\ 
0   & F_s
\end{pmatrix}
+
F_{spin-dependent}
$$
### rearranging... 
# $$ 
F_{spin-dependent} 
=
\frac{1}{2}\left(
\begin{pmatrix}
F_s + F_z & F_x-iF_y \\
F_x + iF_y & F_s-Fz 
\end{pmatrix} 
- 
\begin{pmatrix}
F_s & 0 \\ 
0   & F_s
\end{pmatrix}
\right)
$$
### and finally...
# $$
F_{spin-dependent} 
=
\frac{1}{2}
\begin{pmatrix}
F_z & F_x-iF_y \\
F_x + iF_y & -Fz 
\end{pmatrix} 
$$
# which is what I will build in the next section. 

In [22]:
#taking spin-dependent parts from pauli spinor AO basis and moving into alpha/beta AO basis 

#alpha-alpha block 
topleftsubmat_noscal = 0.5*(fockz_NeX2cNoscal)

#beta-beta block
bottomrightsubmat_noscal = -0.5*(fockz_NeX2cNoscal)

#alpha-beta block 
toprightsubmat_noscal = 0.5*(fockx_NeX2cNoscal-1j*focky_NeX2cNoscal)

#beta-alpha block
bottomleftsubmat_noscal = 0.5*(fockx_NeX2cNoscal+1j*focky_NeX2cNoscal)

In [23]:
#putting fock matrix submatrices together 

#initializing empty matrix with dimensionality double that of scattered 
tot_fock_sd_ao_noscal_x2c = np.zeros((NBasis*2,NBasis*2),dtype=np.complex_)

#placing submatrices into large matrix
tot_fock_sd_ao_noscal_x2c [0:NBasis,0:NBasis] = topleftsubmat_noscal
tot_fock_sd_ao_noscal_x2c [0:NBasis,NBasis:2*NBasis] = toprightsubmat_noscal
tot_fock_sd_ao_noscal_x2c [NBasis:2*NBasis,0:NBasis] = bottomleftsubmat_noscal
tot_fock_sd_ao_noscal_x2c [NBasis:2*NBasis,NBasis:2*NBasis] = bottomrightsubmat_noscal

In [24]:
#sanity check 
#From output file:
print("NBasis = 103")

#times two 
print("2NBasis = " + str(2*NBasis))

#rows
print("number of fock_sd rows: " + str(len(tot_fock_sd_ao_noscal_x2c)))

#columns
print("number of fock_sd columns: " + str(len(tot_fock_sd_ao_noscal_x2c[0])))

#rows = columns = 2*NBasis. That's good!

NBasis = 103
2NBasis = 206
number of fock_sd rows: 206
number of fock_sd columns: 206


In [25]:
tot_fock_sd_ao_noscal_x2c

array([[ 1.59184272e-09+7.86277431e-13j,  1.32469800e-10-7.81347384e-11j,
         1.17071097e-10-7.36471208e-11j, ...,
        -1.17962941e-11-2.69661872e-12j,  1.45234172e-12-3.72978478e-12j,
        -1.24042718e-11-5.76916619e-12j],
       [ 1.40167127e-10+7.94183613e-11j,  3.17715417e-10+1.14471332e-13j,
         1.28602126e-10+2.61445544e-12j, ...,
        -8.50090880e-12-1.13157988e-12j,  8.72118744e-12-1.50183963e-12j,
        -2.39236271e-12+2.68173999e-12j],
       [ 1.18702503e-10+7.38174211e-11j,  1.28593572e-10-2.45739626e-12j,
         8.49749038e-11+8.98279484e-14j, ...,
        -2.02297570e-12-2.01303907e-12j,  1.51434481e-12-1.73330599e-12j,
        -8.23150873e-13+4.68482566e-13j],
       ...,
       [-1.17962941e-11+2.69661872e-12j, -8.50090880e-12+1.13157988e-12j,
        -2.02297570e-12+2.01303907e-12j, ...,
        -3.02698224e-15+5.09439496e-17j,  3.49475219e-15-6.39564593e-15j,
        -3.94649673e-15+9.27638845e-17j],
       [ 1.45234172e-12+3.72978478e-12j,  8.

# Running calculations for x2c-on(4c_DCB)

In [ ]:
#importing bin file from a Kr 1e-X2C with no 2e-SOC (or approx.) with dyall-ae4z basis set.
file_NeX2cNoscal = h5py.File('./data/x2c1enoscal/Ne_dyall_ae4z/neon_x2c_noscal.bin','r')